In [2]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
import time


#np.random.seed(0)
%matplotlib inline

In [3]:
orig_X = np.genfromtxt('data/X_train.txt', delimiter=None)
Y = np.genfromtxt('data/Y_train.txt', delimiter=None)
test_X = np.genfromtxt('data/X_test.txt',delimiter=None)

orig_X,Y = ml.shuffleData(orig_X,Y)

In [19]:
for f in range(orig_X.shape[1]):
    print "Feature #{:3d}".format(f+1)
    print "    min:      {}    min:      {}".format(min(orig_X[:,f]),min(test_X[:,f]))
    print "    max:      {}    max:      {}".format(max(orig_X[:,f]),max(test_X[:,f]))
    print "    mean:     {}    mean:     {}".format(np.mean(orig_X[:,f]),np.mean(test_X[:,f]))
    print "    variance: {}    variance: {}".format(np.var(orig_X[:,f]),np.var(test_X[:,f]))

Feature #  1
    min:      193.5    min:      193.0
    max:      253.0    max:      253.0
    mean:     241.6011037    mean:     241.6357377
    variance: 83.4991711498    variance: 83.0210963348
Feature #  2
    min:      152.5    min:      190.0
    max:      249.0    max:      250.5
    mean:     227.3765713    mean:     227.4260077
    variance: 92.625593125    variance: 91.8753024705
Feature #  3
    min:      214.25    min:      214.25
    max:      252.5    max:      252.5
    mean:     241.5541505    mean:     241.5964344
    variance: 35.2863398033    variance: 35.0991305405
Feature #  4
    min:      152.5    min:      204.42
    max:      252.5    max:      252.5
    mean:     232.82676815    mean:     232.8601144
    variance: 97.6257317486    variance: 97.1176166369
Feature #  5
    min:      10.0    min:      10.0
    max:      31048.0    max:      31048.0
    mean:     3089.923365    mean:     3080.187265
    variance: 15651513.7564    variance: 15431666.549
Feature #  

In [4]:
orig_X = np.delete(orig_X,[4,5,12,13],1)
print orig_X.shape

(200000, 10)


so yeah this is just the reading input part done...

In [5]:
import pandas as pd


from sklearn import preprocessing
X = preprocessing.robust_scale(orig_X)

df_X = pd.DataFrame(X)
df_Y = pd.DataFrame(Y)
print df_X.shape
print df_Y.shape

(200000, 10)
(200000, 1)


In [6]:
df_x = df_X[:2000]
df_y = df_Y[:2000]
print df_x.shape
print df_y.shape

(2000, 10)
(2000, 1)


In [6]:
import seaborn as sns
#sns.pairplot(df_x)

so currently looking at split on x3 and choose one feature from {x1,x2,x9,x8} also drop x13 & x12

In [7]:
cooment = '''
for a in range(13):
    for b in range(a+1,14):
        print "x{} vs x{}".format(a,b)
        %matplotlib inline
        fig,ax = plt.subplots(1,1,figsize=(16,10))
        ax.scatter(X[:,a][Y==0],X[:,b][Y==0],color="b",alpha=0.2,s=5)
        ax.scatter(X[:,a][Y==1],X[:,b][Y==1],color="r",alpha=0.2,s=5)

        plt.show()
%matplotlib inline
'''

In [8]:
#learner = ml.dtree.treeClassify(X,Y,maxDepth=50)

In [7]:
from sklearn import tree
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=6)
clf = tree.DecisionTreeClassifier(max_depth=100)
aucs = []
for train, test in cv.split(df_x, df_y):
    clf = clf.fit(df_x.iloc[train],df_y.iloc[train])
    probs = clf.predict_proba(df_x.iloc[test])
    fpr, tpr, thresholds = roc_curve(df_y.iloc[test], probs[:, 1])
    aucs.append(auc(fpr,tpr))
print aucs

[0.51649012532495253, 0.54469793970434177, 0.52747060877662677, 0.57030613432662092, 0.52460691823899375, 0.56210691823899372]


In [8]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
df_poly_X = poly.fit_transform(df_X)
print df_poly_X.shape

df_poly_x = poly.fit_transform(df_x)
print df_poly_x.shape

print len(poly.get_feature_names())

logisticRegr = LogisticRegression()
selector = RFE(clf, 10, step=1)
selector = selector.fit(df_poly_x,df_y)
#print selector.ranking_

(200000, 66)
(2000, 66)
66


In [11]:
from collections import Counter
for i in range(len(selector.support_)):
    if selector.support_[i]:
        print poly.get_feature_names()[i]

x0 x7
x0 x11
x1 x13
x2 x12
x4 x10
x4 x11
x4 x12
x7 x11
x10^2
x11^2


In [11]:
filtered_X = poly.fit_transform(df_poly_X)
filtered_X = pd.DataFrame(filtered_X)
print filtered_X.shape
print df_poly_X.shape

(200000, 66)


In [12]:
cv = StratifiedKFold(n_splits=6)
aucs = []
for train, test in cv.split(filtered_X, df_Y):
    logisticRegr = logisticRegr.fit(filtered_X.iloc[train],df_Y.iloc[train])
    probs = logisticRegr.predict_proba(filtered_X.iloc[test])
    fpr, tpr, thresholds = roc_curve(df_Y.iloc[test], probs[:, 1])
    aucs.append(auc(fpr,tpr))
print aucs

NameError: name 'filtered_X' is not defined

In [14]:
c = """
cv = StratifiedKFold(n_splits=6)
aucs = []
from sklearn.ensemble import RandomForestClassifier
for j in range(11,31):
    clf = RandomForestClassifier(max_depth=j)
    selector = RFE(clf, 20, step=1)
    selector = selector.fit(df_poly_x,df_y)
    filtered_X = selector.transform(df_poly_X)
    filtered_X = pd.DataFrame(filtered_X)
    print filtered_X.shape
    aucs_j = []
    for train, test in cv.split(filtered_X, df_Y):
        clf = clf.fit(filtered_X.iloc[train],df_Y.iloc[train])
        probs = clf.predict_proba(filtered_X.iloc[test])
        fpr, tpr, thresholds = roc_curve(df_Y.iloc[test], probs[:, 1])
        aucs_j.append(auc(fpr,tpr))
    print aucs_j,j
    aucs.append((np.mean(aucs_j),j))
    if len(aucs) > 1 and aucs[-2]>aucs[-1]:
        break;
print aucs
"""

In [16]:
from sklearn.ensemble import RandomForestClassifier

cv = StratifiedKFold(n_splits=6)
aucs = []

df_poly_orig_X = poly.fit_transform(orig_X)

filtered_X = pd.DataFrame(df_poly_orig_X)
print filtered_X.shape

(200000, 66)


In [ ]:
for j in range(23,40,1):    
    aucs_j = []
    clf = RandomForestClassifier(max_depth=j,min_samples_leaf=6)
    for train, test in cv.split(filtered_X, df_Y):
        clf = clf.fit(filtered_X.iloc[train],df_Y.iloc[train])
        probs = clf.predict_proba(filtered_X.iloc[test])
        fpr, tpr, thresholds = roc_curve(df_Y.iloc[test], probs[:, 1])
        aucs_j.append(auc(fpr,tpr))
    print aucs_j,j
    aucs.append((np.mean(aucs_j),j))
    if len(aucs) > 1 and max(aucs[:-1])[0]-aucs[-1][0] > 0.005:
        break;
print aucs
# best random forest at max_depth=20, min_samples_leaf=6

c:\python27\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[0.7637401646698837, 0.75844335770117843, 0.76470055326128428, 0.76747187322262245, 0.76336183847904282, 0.76058509662710916] 23
[0.76242012788485447, 0.7658151720831281, 0.76412331875927864, 0.77059393822176814, 0.76559463104823622, 0.76219130237393995] 24
[0.76359141448854684, 0.7608420195307033, 0.76694196923567537, 0.76925838089350362, 0.76182773613828791, 0.75839734509476497] 25
[0.76501753935250416, 0.76456595654496806, 0.76669992641076912, 0.76428500128759069, 0.76314546252188276, 0.75768518939408414] 26
[0.76584335119861291, 0.763097359303334, 0.76800997768357837, 0.76748705788574134, 0.76556510254051424, 0.76098528535924559] 27


In [15]:
print max(aucs)

(0.76498227379506378, 24)


In [17]:
df_poly_orig_X = poly.fit_transform(orig_X)
clf = RandomForestClassifier(max_depth=20)
filtered_X = pd.DataFrame(df_poly_orig_X)
print filtered_X.shape

aucs_j = []
for train, test in cv.split(filtered_X, df_Y):
    clf = clf.fit(filtered_X.iloc[train],df_Y.iloc[train])
    probs = clf.predict_proba(filtered_X.iloc[test])
    fpr, tpr, thresholds = roc_curve(df_Y.iloc[test], probs[:, 1])
    aucs_j.append(auc(fpr,tpr))
print aucs_j,j
print mean(aucs_j)

(200000, 120)


c:\python27\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


KeyboardInterrupt: 

In [ ]:
np.mean(aucs_j)